In [ ]:
#necessary libraries for topic-modeling
import base64
import numpy as np
import pandas as pd
# Plotly imports
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
#postgre connection import
import psycopg2
# Other imports
from collections import Counter
from imageio import imread
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from matplotlib import pyplot as plt
%matplotlib inline
# spacy for lemmatization
import spacy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import string
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import random
import io
# import unidecode
import random
import torch
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import re
from pprint import pprint

from keras import utils as np_utils
from tensorflow.keras.utils import to_categorical

# keras module for building LSTM 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

# set seeds for reproducability
import tensorflow
from numpy.random import seed
tensorflow.random.set_seed(2)
seed(1)
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter
!pip install pyspellchecker
import re 
from wordcloud import WordCloud, STOPWORDS
from spellchecker import SpellChecker
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
from collections import Counter
%matplotlib inline
import torch.utils.data
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import os

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning:

The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.



     |████████████████████████████████| 2.7 MB 3.7 MB/s 


In [ ]:
#connecting to the database
conn = psycopg2.connect("dbname = 'postgres' user = 'cpdbstudent' host = 'codd01.research.northwestern.edu' password = 'DataSci4AI'")
cur = conn.cursor()
query = """
    select summary from  data_allegation where summary is not null and summary != ''
"""
outputquery = "COPY ({0}) TO STDOUT WITH CSV HEADER".format(query)
with open('data_attachmentfile.csv', 'w') as f:
    cur.copy_expert(outputquery, f)

In [ ]:
train = pd.read_csv("data_attachmentfile.csv")

In [ ]:
train.head(5)

,summary
0,"On June 1st, 2009, a complaint was registered ..."
1,"On 23 June 2009, a complaint was registered wi..."
2,An incident involving two on-duty Chicago Poli...
3,"On July 28th, 2009 a complaint was registered ..."
4,"On July 31 2009, a complaint was registered wi..."


In [ ]:
corpus = train.copy()

In [ ]:
df = train.copy()
df.head()

,summary
0,"On June 1st, 2009, a complaint was registered ..."
1,"On 23 June 2009, a complaint was registered wi..."
2,An incident involving two on-duty Chicago Poli...
3,"On July 28th, 2009 a complaint was registered ..."
4,"On July 31 2009, a complaint was registered wi..."


In [ ]:
with io.open('/content/data_attachmentfile.csv', encoding="utf-8") as f:
    text = f.read().lower()
text = text.replace("\n", " ")  # We remove newlines chars for nicer display
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Corpus length: 1286518
Total chars: 59
Number of sequences: 428826


In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
epochs = 25
batch_size = 128

for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print()
    print("Generating text after epoch: %d" % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5]:
        print("...Diversity:", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('...Generating with seed: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("...Generated: ", generated)
        print()

3351/3351 [==============================] - 21s 5ms/step - loss: 1.3012

Generating text after epoch: 0
...Diversity: 0.2
...Generating with seed: "red firearm and failed to report the mis"
...Generated:  conduct was alleged that officer b and force and witness and and the complainants and the complainants and that he was arrested and reports and statement to the complainants and that he was arrested that he was able that he was abused the complainants and a false report it was alleged that officer a and failed to report it was alleged that officer b and b tochece officer was intoxicated and and th

...Diversity: 0.5
...Generating with seed: "red firearm and failed to report the mis"
...Generated:  conducted and officer witnesses, and intoxicated. it was alleged that officer a agains that occurred with the independent police review authority (ipra) repartments for a follow hanty and reconding and witnes his hand. the subject. it was alleged that the accused officer was alleged that of

In [ ]:
np.max(x_pred[0])

1.0

In [ ]:
sentence = text[start_index : start_index + maxlen]
print('...Generating with seed: "' + sentence + '"')

...Generating with seed: "pa finds that officer a?s actions did no"


In [ ]:
start_index = random.randint(0, len(text) - maxlen - 1)
for diversity in [0.2, 0.5]:
    print("...Diversity:", diversity)

    generated = ""
    sentence = text[start_index : start_index + maxlen]
    print('...Generating with seed: "' + sentence + '"')

    for i in range(20):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.0
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        sentence = sentence[1:] + next_char
        generated += next_char

    print("...Generated: ", generated)
    print()

...Diversity: 0.2
...Generating with seed: " to the victim and her family members on"
...Generated:   the arment of the a

...Diversity: 0.5
...Generating with seed: " to the victim and her family members on"
...Generated:   the both officers, 



In [ ]:
start_index = random.randint(0, len(text) - maxlen - 1)
for diversity in [0.2, 0.5]:
    print("...Diversity:", diversity)

    generated = ""
    sentence = text[start_index : start_index + maxlen]
    print('...Input Sentence: "' + sentence + '"')
    count = 0
    for i in range(20):
        if count>3:
          break
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.0
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        sentence = sentence[1:] + next_char
        generated += next_char
        if next_char == ' ':
          count += 1

    print("...Autocomplete: ", generated)
    print()

...Diversity: 0.2
...Input Sentence: " battery, and subsequently pleaded guilt"
...Autocomplete:  y of the complainant

...Diversity: 0.5
...Input Sentence: " battery, and subsequently pleaded guilt"
...Autocomplete:  y of the department 



In [ ]:
print("Done!")

Done!
